In [ ]:
!pip install camelot-py

In [5]:
!apt install ghostwriter

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package ghostwriter


In [6]:
!pip install ghostwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 1.8 MB/s 
     |████████████████████████████████| 1.7 MB 11.4 MB/s 
     |████████████████████████████████| 970 kB 38.2 MB/s 
     |████████████████████████████████| 181 kB 42.9 MB/s 
     |████████████████████████████████| 547 kB 48.4 MB/s 
     |████████████████████████████████| 338 kB 57.9 MB/s 
     |████████████████████████████████| 8.3 MB 41.3 MB/s 
     |████████████████████████████████| 146 kB 61.7 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 133 kB 46.8 MB/s 
     |████████████████████████████████| 40 kB 5.3 MB/s 
     |████████████████████████████████| 77 kB 2.7 MB/s 
     |████████████████████████████████| 95 kB 4.2 MB/s 
     |████████████████████████████████| 96 kB 5.0 MB/s 
     |████████████████████████████████| 224 k

* Barra de progresso que será exibida na transformação de pdf para .csv ↓

In [7]:
import zipfile
import os
import camelot 
import pandas as pd
import ghostscript


def progresso(atual, total):
    porcentagem = round(atual / (total / 100), 2)
    return porcentagem

* Extração do anexo 1 do anexos.zip que deve estar no ambiente de execução ↓

In [8]:
with zipfile.ZipFile('anexos.zip', 'r') as anexos_zip:
    anexo_1 = (anexos_zip.namelist())[0]
    anexos_zip.extract(anexo_1)

* O camelot irá ler as páginas especificadas nas variáveis 'pag_inicial' e 'pag_final' e transformar as tabelas em .csv, após cada leitura elas serão concatenadas num dataframe pandas.
* O máximo de otimização que eu consegui nessa parte foi com um 'for' para ler uma por uma, provavelmente há um modo mais eficiente.
* Em seguida as rows das colunas 'OD' e 'AMB' são alteradas no dataframe. Entendi essa tarefa de duas formas, explicitadas no código abaixo↓

In [9]:
master_df = pd.DataFrame()
pag_inicial = 3
pag_final = 200
ler = (pag_final-pag_inicial)+1
for n in range(ler):
    tabelas = camelot.read_pdf(f'{anexo_1}', process_background=True,
                               pages=f'{n + pag_inicial}')
    tabelas.export(f'tabela.csv', f='csv')
    master_df = pd.concat([master_df, pd.read_csv(f'tabela-page-{n + pag_inicial}-table-1.csv')])
    os.remove(f'tabela-page-{n + pag_inicial}-table-1.csv')
    print(f'Criando tabelas: {progresso(n, ler)}% concluído. {n}/{ler}')
print(f'100% concluído. {ler}/{ler}')

# cria um novo pandas dataframe, itera sobre as rows e altera de acordo
master_df.to_csv('tabela_unica.csv', index=False)
df = pd.read_csv("tabela_unica.csv")
correct_df = df.copy()
for index, row in correct_df.iterrows():
    if row['OD'] == 'OD':
        correct_df.loc[index, 'OD'] = 'Seg. Odontológica'
    if row['AMB'] == 'AMB':
        correct_df.loc[index, 'AMB'] = 'Seg. Ambulatorial'

# entendi a tarefa de alterar as células de forma ambígua
# caso o correto seja alterar apenas as colunas, ative abaixo:
'''
correct_df.rename(columns={'OD': 'Seg. Odontológica', 'AMB': 'Seg. Ambulatorial'}, inplace=True)
'''

correct_df.to_csv(r'tabela_unica_fixd.csv', index=False, header=True)
with zipfile.ZipFile('Teste_Pedro_Mendes.zip', 'w', compression=zipfile.ZIP_DEFLATED) as zipado:
    zipado.write('tabela_unica_fixd.csv')
os.remove('tabela_unica.csv')
os.remove('tabela_unica_fixd.csv')
os.remove(f'{anexo_1}')

Criando tabelas: 0.0% concluído. 0/198
Criando tabelas: 0.51% concluído. 1/198
Criando tabelas: 1.01% concluído. 2/198
Criando tabelas: 1.52% concluído. 3/198
Criando tabelas: 2.02% concluído. 4/198
Criando tabelas: 2.53% concluído. 5/198
Criando tabelas: 3.03% concluído. 6/198
Criando tabelas: 3.54% concluído. 7/198
Criando tabelas: 4.04% concluído. 8/198
Criando tabelas: 4.55% concluído. 9/198
Criando tabelas: 5.05% concluído. 10/198
Criando tabelas: 5.56% concluído. 11/198
Criando tabelas: 6.06% concluído. 12/198
Criando tabelas: 6.57% concluído. 13/198
Criando tabelas: 7.07% concluído. 14/198
Criando tabelas: 7.58% concluído. 15/198
Criando tabelas: 8.08% concluído. 16/198
Criando tabelas: 8.59% concluído. 17/198
Criando tabelas: 9.09% concluído. 18/198
Criando tabelas: 9.6% concluído. 19/198
Criando tabelas: 10.1% concluído. 20/198
Criando tabelas: 10.61% concluído. 21/198
Criando tabelas: 11.11% concluído. 22/198
Criando tabelas: 11.62% concluído. 23/198
Criando tabelas: 12.12% c